In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsClassifier
import optuna
import optuna.visualization as vis

n = 9

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):
    ### ========== Shuffle and Split Data ========== ###

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.8, random_state=42)  # 60% training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.8, random_state=42)  # 20% validation, 20% test

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Suggest dimensionality reduction and kernel parameters
    use_pca = trial.suggest_categorical('use_pca', ['True', 'False'])
    n_components_pca = trial.suggest_int('n_pca_components', 1, max(1, X_train_scaled.shape[1]))
    kernel_pca = trial.suggest_categorical('pca_kernel', ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'])

    ### ========== Dimensionality Reduction ========== ###

    if use_pca == 'True':
        pca = KernelPCA(
            kernel=kernel_pca,
            n_components=n_components_pca,
            n_jobs=-1
        )
        X_train_scaled = pca.fit_transform(X_train_scaled)
        X_val_scaled = pca.transform(X_val)
        X_test_scaled = pca.transform(X_test)

    # KNN Classifier parameters
    classifier = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 5, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()

    # Evaluate on the test set
    y_test_pred = classifier.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return val_accuracy

# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

# Visualize the optimization history and parameter importances
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

c:\Users\berma\Documents\Mathematics for Machine Learning\MML-CW\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-11-11 19:27:25,517] A new study created in memory with name: no-name-3fa964d8-b00f-4179-83c4-1a9fe8bcda10
[I 2024-11-11 19:27:25,998] Trial 0 finished with value: 0.5069444444444444 and parameters: {'use_pca': 'True', 'n_pca_components': 2, 'pca_kernel': 'sigmoid', 'n_neighbors': 17, 'weights': 'distance'}. Best is trial 0 with value: 0.5069444444444444.


Test Accuracy: 0.5087


[I 2024-11-11 19:27:26,707] Trial 1 finished with value: 0.5215277777777778 and parameters: {'use_pca': 'True', 'n_pca_components': 5, 'pca_kernel': 'cosine', 'n_neighbors': 9, 'weights': 'uniform'}. Best is trial 1 with value: 0.5215277777777778.


Test Accuracy: 0.5293


[I 2024-11-11 19:27:27,810] Trial 2 finished with value: 0.48194444444444445 and parameters: {'use_pca': 'True', 'n_pca_components': 9, 'pca_kernel': 'sigmoid', 'n_neighbors': 38, 'weights': 'uniform'}. Best is trial 1 with value: 0.5215277777777778.


...

Test Accuracy: 0.9499


[I 2024-11-11 19:27:45,037] Trial 96 finished with value: 0.9506944444444444 and parameters: {'use_pca': 'False', 'n_pca_components': 7, 'pca_kernel': 'linear', 'n_neighbors': 5, 'weights': 'distance'}. Best is trial 11 with value: 0.9506944444444444.
[I 2024-11-11 19:27:45,088] Trial 97 pruned. 
[I 2024-11-11 19:27:45,188] Trial 98 pruned. 


Test Accuracy: 0.9521


[I 2024-11-11 19:27:45,305] Trial 99 finished with value: 0.9486111111111111 and parameters: {'use_pca': 'False', 'n_pca_components': 8, 'pca_kernel': 'rbf', 'n_neighbors': 6, 'weights': 'distance'}. Best is trial 11 with value: 0.9506944444444444.


Test Accuracy: 0.9499
Best hyperparameters: {'use_pca': 'False', 'n_pca_components': 7, 'pca_kernel': 'linear', 'n_neighbors': 5, 'weights': 'distance'}
Best value: 0.9506944444444444
